In [1]:
NODE_COUNT = 8
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstellar_DFL_18_07_2024_12_59_54/metrics/participant_"
date_str = "18/07/2024 10:59:54	" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/FULL/fedstellar_DFL_18_07_2024_12_59_54/metrics/participant_0
   step      value                     timestamp                    tag
0   143  94.300003 2024-07-18 11:02:17.548973824  Resources/CPU_percent
1   153   5.200000 2024-07-18 11:02:27.567700736  Resources/CPU_percent
2   163   2.500000 2024-07-18 11:02:37.587161600  Resources/CPU_percent
3   173   2.000000 2024-07-18 11:02:47.605036800  Resources/CPU_percent
4   183   2.300000 2024-07-18 11:02:57.623910656  Resources/CPU_percent
2024-07-18 11:24:20 2024-07-18 11:00:05
                 step     value                     timestamp
tag                                                          
Test/Accuracy   13065  0.820015 2024-07-18 11:24:13.408886272
Test/F1Score    13065  0.796834 2024-07-18 11:24:13.408917504
Test/Loss       13065  0.662371 2024-07-18 11:24:13.408858112
Test/Precision  13065  0.818912 2024-07-18 11:24:13.408897280
Test/Recall     13065  0.820015 2024-07-18 11

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,13065,0.820015,2024-07-18 11:24:13.408886272,participant_0
Test/F1Score,13065,0.796834,2024-07-18 11:24:13.408917504,participant_0
Test/Loss,13065,0.662371,2024-07-18 11:24:13.408858112,participant_0
Test/Precision,13065,0.818912,2024-07-18 11:24:13.408897280,participant_0
Test/Recall,13065,0.820015,2024-07-18 11:24:13.408907520,participant_0
Test/Accuracy,11008,0.897124,2024-07-18 11:24:16.972731392,participant_1
Test/F1Score,11008,0.874263,2024-07-18 11:24:16.972759552,participant_1
Test/Loss,11008,0.812031,2024-07-18 11:24:16.972700160,participant_1
Test/Precision,11008,0.889215,2024-07-18 11:24:16.972741888,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_36528/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,11517.375,0.8318
Test/F1Score,11517.375,0.8064
Test/Loss,11517.375,0.9102
Test/Precision,11517.375,0.8342
Test/Recall,11517.375,0.8318


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_36528/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,952.17,0.04
Test/F1Score,952.17,0.04
Test/Loss,952.17,0.38
Test/Precision,952.17,0.04
Test/Recall,952.17,0.04


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_36528/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,11517.375,0.8318
Test/F1Score,11517.375,0.8064
Test/Loss,11517.375,0.9102
Test/Precision,11517.375,0.8342
Test/Recall,11517.375,0.8318
